In [1]:
# Install our (local, for now) pip package 
!python3 -m pip install -e C://Users/t-jorhen/Documents/GitHub/find-tune

In [1]:
import mlos

mlos.set_project_dir('C://Users/t-jorhen/Documents/GitHub/sqlite/.mlos')
mlos.checkenv(using_azure=True)

project = (
    mlos
    .create_project_from_git_info()
    .with_backend(mlos.backends().azure())
    .with_build_options(mlos.build_options())
)

project.attach(db_only=True)

- Verifying environment...
   ✓ Environment verification complete
- Creating project from current git repository...
   - Getting project name: 
      + project-name:='https://github.com/jjhenkel/sqlite'
   - Getting project commit hash: 
      + project-commit:='ff0c42866cee7e9b8b84a31266aa744ebe7d8bcd'
- Ensuring infrastructure is allocated...
   ✓ Found resource group 'FindTune-EUS'
⠧ Provisioning 'findtune-az-autogen-vnet-140269'   ✓ Using existing vnet: 'findtune-az-autogen-vnet-801663' (['10.0.0.0/16'])
   ✓ Using existing subnet: 'findtune-az-autogen-subnet-806943' (10.0.0.0/24)
   ✓ Using existing ip: 'findtune-az-autogen-ip-ft-db-211043' (52.226.50.179)
   ✓ Using existing vm: 'findtune-az-autogen-vm-ft-db-500381'
- Attaching...
⠧ Starting 'findtune-az-autogen-vm-ft-db-500381'   ✓ 'findtune-az-autogen-vm-ft-db-500381' is running!
   ✓ Skipping DB create. Database 'targets-sqlite' already exists!
   ✓ Attached. Ready to run queries


In [2]:
project.ingest()

In [3]:
project.poll_status()

In [4]:
project.detach(keep_db=True)

In [9]:
results = (project
  .query()
  .records()
  .name_contains("config")
  .make_visualizable()
  .fields()
  .select_usages()
  .from_files_matching('/sqlite3.c')
  .execute()
).results

USES_GLOBAL = float(len([ 
  x for x in results if 'Global' in x['clang_source_range_as_text']
]))

TOTAL = float(len(results))

print("Uses sqlite3GLOBAL... in {}/{} ({:.2%}) of places".format(
  USES_GLOBAL, TOTAL, USES_GLOBAL / TOTAL
))

Uses sqlite3GLOBAL... in 159.0/167.0 (95.21%) of places


In [8]:
(project
  .query()
  .records()
  .name_contains("cache")
  .field_references()
  .restrict_to_assignments(
    where_rhs=mlos.filters().arithmetic()
  )
  .make_visualizable()
  .select()
  .execute()
  .display()
)

In [5]:
(
  project
    .query()
    .records()
    .comment_semantically_similar_to("page cache")
    .make_visualizable()
    .select()
    .execute()
    .display()
)

In [7]:
(project
  .query()
  .records()
  .fields()
  .integers()
  .comment_semantically_similar_to("desired size")
  .make_visualizable()
  .select()
  .execute()
  .display()
)

In [8]:
(project
    .query()
    .records()
    .name_contains("sqlite3_pcache_page")
    .fields()
    .make_visualizable()
    .select_usages()
    .execute()
    .display()
)

In [17]:
(project.query().ifs().make_visualizable().select().limit(10).execute().display())